# Question 3 Vision

### Overview:

One of the software division’s primary tasks is identifying and localizing ODLCs as mentioned previously. As written in the reference, these ODLCs are composed of a unique color, character, and shape that must be classified from 100ft in the air. However, in order to train models with high accuracy, we would need to collect thousands of images over 100+ hours of flight time. We, unfortunately, cannot reasonably perform this task before the competition deadline. So, in order to have both well-trained models within timeframes, we use generation scripts which can produce thousands of images within seconds that can somewhat match the quality and precision of the camera in black and white. 

### Your Task:

- Write generator script(s) for shape and/or character.
- Train a YOLOv8n model on this data (hint: is there a way to not manually label every image?).
- Correctly recognize the shape and character of the ODLC.

### Constraints: 

Character - At SUAS, all alphanumeric characters are allowed, but for simplicity, it can be assumed that all test cases are uppercase letters (ABCDEFGHIJKLMNOPQRSTUVWXYZ)
Shape - The only valid shapes are circle, semicircle, quarter circle, triangle, rectangle, pentagon, star, and cross.

### Restrictions:

Don’t use non-standard libraries except PIL, ultralytics, numpy, and opencv-python.

P.S: If it is taking more than 5 hours to train a YOLOv8n model, reduce the number of training images. I am mostly trying to learn about your problem-solving process rather than the actual results.

### File Input Format: 
The .in file will consist of a square image with random dimensions, and the ODLC is guaranteed to be in the image however, it may not be in the center.

### File Output Format: 
Output the shape and character respectively separated by spaces. When submitting the deliverables for this problem, include the YOLOv8 .pt model along with the models, labels, and all YOLOv8 created (like runs and predict) directories.

### Draw a Circle

In [1]:
from PIL import Image, ImageDraw, ImageFont
import random

# fix the image size
image_size = 360

# Draw the shape that fits 70-80% of the image
shape_size = image_size * random.randint(70, 80) // 100


In [2]:
# Draw a circle
def draw_ellipse(draw, center, size, outline="black", width=10):
    size = size // 2
    xy = (center[0] - size, center[1] - size, center[0] + size, center[1] + size)
    draw.ellipse(xy, outline=outline, width=width)
    return xy

In [3]:
# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

shape_bb = draw_ellipse(draw, [image_size//2, image_size//2], shape_size)
print("shape bounding box =", shape_bb)

draw.rectangle(shape_bb, outline="blue")
img.show()

shape bounding box = (53, 53, 307, 307)


### Draw a Character

In [4]:
# Draw the character that fits inside the shape
def draw_character(draw, character, bbox, center_ratio = 0.5, char_ratio = 0.75, font_type = "Arial Unicode", debug=False):
    from PIL import ImageFont

    if debug: print("\nDraw Character")
    
    # Choose a font size that fits inside the shape's bounding box
    font_size = int(min(bbox[2] - bbox[0], bbox[3] - bbox[1]) * char_ratio)
    if debug: print("font_size = ", font_size)
    font = ImageFont.truetype(f"/Library/Fonts/{font_type}.ttf", font_size)
    
    center_x = bbox[0] + (bbox[2] - bbox[0]) // 2
    center_y = bbox[1] + int((bbox[3] - bbox[1]) * center_ratio)
    if debug: print("center x, y =", center_x, center_y)

    # Calculate text size and position the character inside the shape
    text_bb = draw.textbbox((center_x, center_y), character, font=font)
    if debug: print("text_bbox =", text_bb)

    text_len = text_bb[2] - text_bb[0]
    text_height = text_bb[3] - text_bb[1]
    if debug: 
        print(f"text (length, height) = (text_len, text_height)")

    text_left = center_x - text_len // 2
    text_top = center_y - text_height
    text_bb = draw.textbbox((text_left, text_top), character, font=font)
    if debug: print("new text_bbox =", text_bb)

    draw.text((text_left, text_top), character, font=font, fill='black')

    return text_bb


In [5]:
# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

circle_bb = draw_ellipse(draw, [image_size//2, image_size//2], shape_size)
print("circle bounding box =", circle_bb)

char_bb = draw_character(draw, "A", circle_bb, debug=True)
print("text bounding box = ", char_bb)

draw.rectangle(circle_bb, outline="blue")
draw.rectangle(char_bb, outline="red")
img.show()

circle bounding box = (53, 53, 307, 307)

Draw Character
font_size =  190
center x, y = 180 180
text_bbox = (179, 248, 307, 384)
text (length, height) = (text_len, text_height)
new text_bbox = (115, 112, 243, 248)
text bounding box =  (115, 112, 243, 248)


### Draw a Triangle

In [6]:
def calc_bounding_box(points: list):
    """Return bounding box for a list of points of (x, y)"""
    X = [p[0] for p in points]
    Y = [p[1] for p in points]
    return (min(X), min(Y), max(X), max(Y))

In [7]:
def draw_triangle(draw, center, size, outline="black", width=10):
    size = size // 2
    points = [
        (center[0], center[1] - size), 
        (center[0] - size, center[1] + size), 
        (center[0] + size, center[1] + size)
    ]
    draw.polygon(points, outline=outline, width=width)
    return calc_bounding_box(points)

In [8]:
# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

shape_bb = draw_triangle(draw, [image_size//2, image_size//2], shape_size)
print("shape bounding box =", shape_bb)

char_bb = draw_character(draw, "U", shape_bb, center_ratio=0.66, char_ratio=0.6)
print("text bounding box = ", char_bb)

# draw bbox
draw.rectangle(shape_bb, outline="blue")
draw.rectangle(char_bb, outline="red")
img.show()

shape bounding box = (53, 53, 307, 307)
text bounding box =  (125, 163, 235, 274)


### Draw Other Shapes

In [9]:
def draw_semicircle(draw, center, size, outline="black", width=10):
    size = size // 2
    xy = (center[0] - size, center[1] - size // 2, center[0] + size, center[1] + size * 3 // 2)
    draw.pieslice(xy, 180, 360, outline=outline, width=width)
    bbox = (center[0] - size, center[1] - size // 2, center[0] + size, center[1] + size // 2)
    return bbox

In [10]:
# Test semicircle
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
shape_bb = draw_semicircle(draw, [image_size//2, image_size//2], shape_size)
# draw bbox
draw.rectangle(shape_bb, outline="blue")
img.show()

In [11]:
def draw_quarter_circle(draw, center, size, outline="black", width=10):
    size = size // 3
    xy = (center[0] -  size * 3, center[1] - size * 3, center[0] + size, center[1] + size)
    draw.pieslice(xy, 0, 90, outline=outline, width=width)
    bbox = (center[0] - size, center[1] - size, center[0] + size, center[1] + size)
    return bbox

In [12]:
# Test quarter circle
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
shape_bb = draw_quarter_circle(draw, [image_size//2, image_size//2], shape_size)
# draw bbox
draw.rectangle(shape_bb, outline="blue")
img.show()

In [13]:
def draw_rectangle(draw, center, size, outline="black", width=10, hw_ratio=0.66):
    w = size // 2
    h = int(w * hw_ratio)
    xy = (center[0] - w, center[1] - h, center[0] + w, center[1] + h)
    draw.rectangle(xy, outline=outline, width=width)
    return xy

In [14]:
# Test rectangle
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
shape_bb = draw_rectangle(draw, [image_size//2, image_size//2], shape_size)
# draw bbox
draw.rectangle(shape_bb, outline="blue")
img.show()

In [15]:
import math

def draw_pentagon_1(draw, center, size, rotation=0, outline="black", width=10):
    radius = size // 2
    draw.regular_polygon((center[0], center[1], radius), n_sides=5, rotation=rotation, outline=outline, width=width)
    if rotation == 0:
        y_max = center[1] + radius * math.sqrt(3/4)
    else:
        y_max = center[1] + radius
    points = [
        (center[0], center[1] - radius),
        (center[0] - radius // 2, y_max),
        (center[0] + radius // 2, y_max),
        (center[0] - radius, center[1]),
        (center[0] + radius, center[1]),
    ]
    return calc_bounding_box(points)

In [34]:
def draw_pentagon_2(draw, center, size, outline="black", width=10):
    """Draw pentagon with draw.polygon"""
    radius = size // 2
    angle = 360 / 5
    points = []

    def angle_cos_sin(angle):
        return math.cos(math.radians(angle)), math.sin(math.radians(angle))

    for i in range(5):
        cos_angle, sin_angle = angle_cos_sin(angle * i)
        x = int(center[0] + radius * cos_angle)
        y = int(center[1] + radius * sin_angle)
        points.append((x, y))

    draw.polygon(points, outline=outline, width=width)

    return calc_bounding_box(points)

In [32]:
def draw_pentagon(draw, center, size, outline="black", width=10):
    return draw_pentagon_2(draw, center, size, outline, width)

In [35]:
# Test pentagon
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
shape_bb = draw_pentagon(draw, [image_size//2, image_size//2], shape_size)
# draw bbox
draw.rectangle(shape_bb, outline="blue")
img.show()

#### Draw Star

In [17]:
# placeholder for star
def draw_star_old(draw, center, size, color="black", width=10):
    return draw_pentagon(draw, center, size, rotation=30, color=color, width=width)

In [18]:
def draw_star(draw, center, size, fill="white", outline="black", width=10):
    angle = 360 / 5
    radius = size // 2
    inner_radius = radius / 2  # Adjust for inner points
    points = []

    def angle_cos_sin(angle):
        return math.cos(math.radians(angle)), math.sin(math.radians(angle))

    # get five pairs of (outer, inner) points
    for i in range(5):
        outer_cos, outer_sin = angle_cos_sin(angle * i)
        outer_x = int(center[0] + radius * outer_cos)
        outer_y = int(center[1] + radius * outer_sin)
        points.append((outer_x, outer_y))

        inner_cos, inner_sin = angle_cos_sin(angle * i + angle / 2)
        inner_x = int(center[0] + inner_radius * inner_cos)
        inner_y = int(center[1] + inner_radius * inner_sin)
        points.append((inner_x, inner_y))

    draw.polygon(points, fill=fill, outline=outline, width=width)
    return calc_bounding_box(points)


In [19]:
# Test star
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
shape_bb = draw_star(draw, [image_size//2, image_size//2], shape_size)
draw.rectangle(shape_bb, outline="blue")
img.show()

In [20]:
def draw_cross(draw, center, size, outline="black", width=5):
    size = size // 3
    draw_rectangle(draw, (center[0], center[1]), size * 3, hw_ratio=1/3, outline=outline, width=width)
    draw_rectangle(draw, (center[0], center[1]), size, hw_ratio=3, outline=outline, width=width)
    draw_rectangle(draw, center, size, hw_ratio=1, outline="white", width=10)
    w = size * 3 // 2
    bbox = (center[0] - w, center[1] - w, center[0] + w, center[1] + w)
    return bbox

In [21]:
# Test cross
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
shape_bb = draw_cross(draw, [image_size//2, image_size//2], shape_size)
draw.rectangle(shape_bb, outline="blue")
img.show()

### Test All Shapes with Text

In [22]:
def draw_shape(draw, shape, center, size, outline="black", width=10):
    if shape == "circle":
        return draw_ellipse(draw, center, size, outline=outline, width=width)
    elif shape == "triangle":
        return draw_triangle(draw, center, size, outline=outline, width=width)
    elif shape == "semicircle":
        return draw_semicircle(draw, center, size, outline=outline, width=width)
    elif shape == "quarter_circle":
        return draw_quarter_circle(draw, center, size, outline=outline, width=width)
    elif shape == "rectangle":
        return draw_rectangle(draw, center, size, outline=outline, width=width)
    elif shape == "pentagon":
        return draw_pentagon(draw, center, size, outline=outline, width=width)
    elif shape == "star":
        return draw_star(draw, center, size, outline=outline, width=width)
    elif shape == "cross":
        return draw_cross(draw, center, size, outline=outline, width=width)
    else:
        print(f"{shape} is not supported. SKIP")

In [23]:
def draw_shape_with_letter(shape, letter, text_ratio=0.75, outline="black", width=10, debug=False):
    # Create a blank image
    img = Image.new('RGB', (image_size, image_size), color='white')
    draw = ImageDraw.Draw(img)

    if debug:
        print("Draw", shape, letter)
    
    shape_bb = draw_shape(draw, shape, [image_size//2, image_size//2], shape_size, outline=outline, width=width)
    if debug:
        print("shape bounding box =", shape_bb)
        draw_rectangle(shape_bb, outline="blue")

    center_ratio = 0.66 if shape == "triangle" else 0.5
    text_bb = draw_character(draw, letter, shape_bb, center_ratio=center_ratio, char_ratio=text_ratio)
    if debug:
        print("text bounding box = ", text_bb)
        draw_rectangle(text_bb, outline="red")

    img.show()

In [24]:
# Test with various shapes
shapes = ['circle', 'semicircle', 'quarter_circle', 'triangle', 'rectangle', 'pentagon', 'star', 'cross']
letters = ['A', 'U', 'L', 'J']
for shape in shapes[-2:-1]:
    for letter in letters:
        draw_shape_with_letter(shape, letter, text_ratio=0.5)

### Rotate Image

In [25]:
import numpy as np

def rotate_points(points, angle, center=(0, 0)):
    """Rotates a list of points around a given center."""
    angle = -angle
    cos_theta = np.cos(np.radians(angle))
    sin_theta = np.sin(np.radians(angle))
    rotated_points = []
    for x, y in points:
        x_diff = x - center[0]
        y_diff = y - center[1]
        x_new = x_diff * cos_theta - y_diff * sin_theta
        y_new = x_diff * sin_theta + y_diff * cos_theta
        x_new += center[0]
        y_new += center[1]
        rotated_points.append((x_new, y_new))
    return rotated_points

In [26]:
points = [(90, 90), (90, 270), (270, 90), (270, 270)]
rotated_points = rotate_points(points, 45, (180, 180))
print(rotated_points)
print(calc_bounding_box(rotated_points))

[(52.72077938642144, 180.0), (180.0, 307.27922061357856), (180.0, 52.72077938642144), (307.27922061357856, 180.0)]
(52.72077938642144, 52.72077938642144, 307.27922061357856, 307.27922061357856)


In [27]:
def rotate_bbox(bbox, angle, img_size):
    import numpy as np
    
    print("\nRotate")
    print("input bbox =", bbox)

    img_center = (img_size / 2, img_size / 2)
    print(f"img_center = {img_center}")

    # Convert the four points of bbox
    x_min, y_min, x_max, y_max = bbox
    points = [(x_min, y_min), (x_min, y_max), (x_max, y_min), (x_max, y_max)]
    new_bbox = calc_bounding_box(rotate_points(points, angle, img_center))
    print(f"rotated bbox = {new_bbox}")

    return new_bbox

#### Test Rotate

In [28]:
import numpy as np

image_size = 360
shape_size = image_size * 0.5
print(f"image_size = {image_size}, shape_size = {shape_size}")

# Create a blank image
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)

offset = int(np.random.uniform(-image_size//10, image_size//10))
print("offset =", offset)
center = [image_size//2 + offset, image_size//2 + offset]
shape_bb = draw_quarter_circle(draw, center, shape_size)
print("shape bounding box =", shape_bb)

char_ratio = random.randint(40, 70) / 100
print("char_ratio =", char_ratio)
char_bb = draw_character(draw, "U", shape_bb, center_ratio=0.66, char_ratio=char_ratio)
print("text bounding box = ", char_bb)
#img.show()

# rotate
angle = 45
img = img.rotate(angle, expand=False, fillcolor='white')
img.show()
rotated_shape_bbox = rotate_bbox(shape_bb, angle, image_size)
rotated_char_bbox = rotate_bbox(char_bb, angle, image_size)

# draw bbox
img = Image.new('RGB', (image_size, image_size), color='white')
draw = ImageDraw.Draw(img)
draw.rectangle(rotated_shape_bbox, outline="blue")
draw.rectangle(rotated_char_bbox, outline="red")
img.show()

image_size = 360, shape_size = 180.0
offset = -1
shape bounding box = (119.0, 119.0, 239.0, 239.0)
char_ratio = 0.48
text bounding box =  (159.0, 177.0, 200.0, 218.0)

Rotate
input bbox = (119.0, 119.0, 239.0, 239.0)
img_center = (180.0, 180.0)
rotated bbox = (93.73297269524122, 95.1471862576143, 263.4386001800126, 264.8528137423857)

Rotate
input bbox = (159.0, 177.0, 200.0, 218.0)
img_center = (180.0, 180.0)
rotated bbox = (163.02943725152286, 163.73654403270942, 221.01219330881975, 221.7193000900063)


### Convert bounding box to YOLO format

In [29]:
# Normalize bounding box for the YOLO label format
def normalize_bbox(bb, img_size):
    x_center = (bb[0] + bb[2]) / 2.0 / img_size
    y_center = (bb[1] + bb[3]) / 2.0 / img_size
    width = (bb[2] - bb[0]) / img_size
    height = (bb[3] - bb[1]) / img_size
    return x_center, y_center, width, height

In [30]:
# Test
print(normalize_bbox(rotated_shape_bbox, image_size))
print(normalize_bbox(rotated_char_bbox, image_size))

(0.49607162899340806, 0.5, 0.4714045207910316, 0.4714045207910317)
(0.5333911535560314, 0.5353553390593273, 0.16106321127026912, 0.16106321127026912)
